In [3]:
from datetime import date
from jugaad_data.nse import bhavcopy_save,bhavcopy_index_save,bhavcopy_fo_save
import pandas as pd
from jugaad_data.holidays import holidays
import sys,os
import mrigutilities as mu

eq_dir = os.path.join('..','data','input')
eq_processed_dir = os.path.join('..','data','processed')


date_range = pd.bdate_range(start='04/08/2024', end = '04/09/2024', 
                         freq='C', holidays = holidays(2024,12))

dates = [x.date() for x in date_range]

print(dates)


[datetime.date(2024, 4, 8), datetime.date(2024, 4, 9)]


In [4]:
from random import randint
import time

for dates in dates:
     try:
        bhavcopy_save(dates, eq_dir)
        bhavcopy_index_save(dates, eq_dir)
        bhavcopy_fo_save(dates, eq_dir)
        time.sleep(randint(1,4)) #adding random delay of 1-4 seconds

#      except (ConnectionError, ReadTimeoutError) as e:
     except:
        time.sleep(10) #stop program for 10 seconds and try again.
        try:
            bhavcopy_save(dates, eq_dir)
            bhavcopy_index_save(dates, eq_dir)
            bhavcopy_fo_save(dates, eq_dir)
            time.sleep(randint(1,4))
        except:
            print(f'{dates}: File not Found')

2024-04-08: File not Found
2024-04-09: File not Found


In [4]:
import requests
from io import StringIO

urls = {'NIFTY 100' : 'https://nsearchives.nseindia.com/content/indices/ind_nifty100list.csv',
        'NIFTY 50' : 'https://nsearchives.nseindia.com/content/indices/ind_nifty50list.csv'
       }
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0'}

result = {}
# print(headers)
for key in urls.keys():
    req = requests.get(urls[key], headers=headers)
    # print(req)
    data = StringIO(req.text)
    df = pd.read_csv(data)
    result[key] = str(df['Symbol'].tolist())
engine = mu.sql_engine()
date = engine.execute("select max(date) from stock_history where symbol='NIFTY 50'").fetchall()[0][0]
disable_sql = "alter table stock_history disable trigger return_trigger"
enable_sql = "alter table stock_history enable trigger return_trigger"
engine.execute(disable_sql)
print(str(date).replace('-',''))
print("Populating Index Constituent")
# usql =  "update stock_history set index_members = %s where symbol = %s and date = %s"
# engine.execute(usql,(result['NIFTY 50'],'NIFTY 50',(str(date).replace('-',''))))
for key in result.keys():
    usql = ""
    usql =  "update stock_history set index_members = %s where symbol = %s and date = %s"
    engine.execute(usql,(result[key],key,(str(date).replace('-',''))))
# n50 = pd.read_csv(n50_url)
# n100 = pd.read_csv(n100_url)

# print(n50)

20240403
Populating Index Constituent


In [20]:
import json
nifty_100 = engine.execute("select index_members from stock_history where symbol='NIFTY 100' and index_members is not NULL order by date desc limit 1").fetchall()[0][0]
nifty_100 = nifty_100.strip('][').split(', ')
nifty_100 = [x[1:-1] for x in nifty_100]
nifty_100

['ABB',
 'ADANIENSOL',
 'ADANIENT',
 'ADANIGREEN',
 'ADANIPORTS',
 'ADANIPOWER',
 'ATGL',
 'AMBUJACEM',
 'APOLLOHOSP',
 'ASIANPAINT',
 'DMART',
 'AXISBANK',
 'BAJAJ-AUTO',
 'BAJFINANCE',
 'BAJAJFINSV',
 'BAJAJHLDNG',
 'BANKBARODA',
 'BERGEPAINT',
 'BEL',
 'BPCL',
 'BHARTIARTL',
 'BOSCHLTD',
 'BRITANNIA',
 'CANBK',
 'CHOLAFIN',
 'CIPLA',
 'COALINDIA',
 'COLPAL',
 'DLF',
 'DABUR',
 'DIVISLAB',
 'DRREDDY',
 'EICHERMOT',
 'GAIL',
 'GODREJCP',
 'GRASIM',
 'HCLTECH',
 'HDFCBANK',
 'HDFCLIFE',
 'HAVELLS',
 'HEROMOTOCO',
 'HINDALCO',
 'HAL',
 'HINDUNILVR',
 'ICICIBANK',
 'ICICIGI',
 'ICICIPRULI',
 'ITC',
 'IOC',
 'IRCTC',
 'IRFC',
 'INDUSINDBK',
 'NAUKRI',
 'INFY',
 'INDIGO',
 'JSWSTEEL',
 'JINDALSTEL',
 'JIOFIN',
 'KOTAKBANK',
 'LTIM',
 'LT',
 'LICI',
 'M&M',
 'MARICO',
 'MARUTI',
 'NTPC',
 'NESTLEIND',
 'ONGC',
 'PIDILITIND',
 'PFC',
 'POWERGRID',
 'PNB',
 'RECLTD',
 'RELIANCE',
 'SBICARD',
 'SBILIFE',
 'SRF',
 'MOTHERSON',
 'SHREECEM',
 'SHRIRAMFIN',
 'SIEMENS',
 'SBIN',
 'SUNPHARMA',
 'TVS